# *Modern Deep Learning for Tabular Data*, Chapter 7

**Tree-Based Deep Learning Models**

This notebook contains the complementary code discussed in Chapter 7 of *Modern Deep Learning for Tabular Data*.

External Kaggle links to datasets used in this notebook:
- [UCI Forest Cover Type Dataset](https://www.kaggle.com/datasets/uciml/forest-cover-type-dataset)

You can download these datasets from Kaggle, or import these notebooks into Kaggle and connect them internally.

In [ ]:
# data management
import numpy as np                   # for linear algebra
import pandas as pd                  # for tabular data manipulation and processing

# machine learning
import sklearn                       # for data prep and classical ML
import tensorflow as tf              # for deep learning
from tensorflow import keras         # for deep learning
import keras.layers as L             # for easy NN layer access
import torch                         # for implementation regarding PyTorch

# data visualization and graphics
import matplotlib.pyplot as plt      # for visualization fundamentals
import seaborn as sns                # for pretty visualizations
import cv2                           # for image manipulation

# misc
from tqdm.notebook import tqdm       # for progress bars
import math                          # for calculation
import sys                           # for system manipulation
import os                            # for file manipulation
from functools import reduce         # for custom implementation

---

## Tree Inspired Networks
We will explore each research-backed approach discussed in the book through concrete examples

### Deep Neural Decision Trees


In [ ]:
from sklearn.datasets import load_iris

data = load_iris()
X = np.array(data.data)
X = torch.from_numpy(X.astype(np.float32))
y = torch.from_numpy(np.array(data.target))

We will use a custom implementation of DNDT through PyTorch

In [ ]:
def torch_kron_prod(a, b):
    res = torch.einsum('ij,ik->ijk', [a, b])
    res = torch.reshape(res, [-1, np.prod(res.shape[1:])])
    return res
def torch_bin(x, cut_points, temperature=0.1):
    # x is a N-by-1 matrix (column vector)
    # cut_points is a D-dim vector (D is the number of cut-points)
    # this function produces a N-by-(D+1) matrix, each row has only one element being one and the rest are all zeros
    D = cut_points.shape[0]
    W = torch.reshape(torch.linspace(1.0, D + 1.0, D + 1), [1, -1])
    cut_points, _ = torch.sort(cut_points)  # make sure cut_points is monotonically increasing
    b = torch.cumsum(torch.cat([torch.zeros([1]), -cut_points], 0),0)
    h = torch.matmul(x, W) + b
    res = torch.exp(h-torch.max(h))
    res = res/torch.sum(res, dim=-1, keepdim=True)
    return h

def nn_decision_tree(x, cut_points_list, leaf_score, temperature=0.1):
    # cut_points_list contains the cut_points for each dimension of feature
    leaf = reduce(torch_kron_prod,
                  map(lambda z: torch_bin(x[:, z[0]:z[0] + 1], z[1], temperature), enumerate(cut_points_list)))
    return torch.matmul(leaf, leaf_score)

We will define some sepcific hyperparameters of DNDT

In [ ]:
num_cut = [2]*4  # 4 features with 1 cut points each
num_leaf = np.prod(np.array(num_cut) + 1) # number of leaf node
num_class = 3
# randomly initialize cutpoints
cut_points_list = [torch.rand([i], requires_grad=True) for i in num_cut]
leaf_score = torch.rand([num_leaf, num_class], requires_grad=True)
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cut_points_list + [leaf_score], lr=0.001)

In [ ]:
# training DNDT
from sklearn.metrics import accuracy_score

for i in range(2000):
    optimizer.zero_grad()
    y_pred = nn_decision_tree(X, cut_points_list, leaf_score, temperature=0.05)
    loss = loss_function(y_pred, y)
    loss.backward()
    optimizer.step()
    if (i+1) % 100 == 0:
        print(f"EPOCH {i} RESULTS")
        print(accuracy_score(np.array(y), np.argmax(y_pred.detach().numpy(), axis=1)))

In [ ]:
# prediction and evaluation
y_pred = nn_decision_tree(X, cut_points_list, leaf_score, temperature=0.01)
y_pred = np.argmax(y_pred.detach().numpy(), axis=1)
print(accuracy_score(np.array(y), y_pred))

### Soft Decision Tree Regressors

Custom model (does not work, but illustrates the idea):

In [ ]:
NUM_SAMPLES = 50_000
INPUT_DIM = 4
x, y = [], []

target_func = lambda x: float((x[0] & x[1]) | x[2]) 
for i in tqdm(range(NUM_SAMPLES)):
    x.append(np.random.choice([0, 1], size=(INPUT_DIM,)))
    y.append(target_func(x[-1]))
x = np.array(x)
y = np.array(y).reshape(-1, 1)

In [ ]:
MAX_DEPTH = 4
LATENT_DIM = 8

inp = L.Input((INPUT_DIM,))
outputs = []
for node in range(sum([2**i for i in range(MAX_DEPTH + 1)])):
    mid = L.Dense(LATENT_DIM, activation='relu')(inp)
    outputs.append(L.Dense(1, activation='sigmoid')(mid))
model = keras.models.Model(inputs=inp, outputs=outputs)

In [ ]:
index = 0

class Node():
    def __init__(self):
        global index
        self.index = index
        self.left = None
        self.right = None
        index += 1

def add_nodes(depths_left):
    curr = Node()
    if depths_left != 0:
        curr.left = add_nodes(depths_left - 1)
        curr.right = add_nodes(depths_left - 1)
    return curr

root = add_nodes(MAX_DEPTH)
def get_outs(root, y_pred):
    if not root.left and not root.right: # is a leaf
        return [y_pred[root.index]]
    lefts = get_outs(root.left, y_pred)
    lefts = [y_pred[root.index] * prob for prob in lefts]
    rights = get_outs(root.right, y_pred)
    rights = [(1-y_pred[root.index]) * prob for prob in rights]
    return lefts + rights

from tensorflow.keras.losses import binary_crossentropy as bce
NUM_OUT = tf.constant(2**MAX_DEPTH, dtype=tf.float32)
def custom_loss(y_true, y_pred):
    outputs = get_outs(root, y_pred)
    return tf.math.divide(tf.add_n([bce(y_true, out) for out in outputs]), NUM_OUT)
model.compile(optimizer='adam', loss=custom_loss)
import tensorflow as tf
avg_loss = tf.keras.metrics.Mean('loss', dtype=tf.float32)
class custom_fit(tf.keras.Model):
    def train_step(self, data):
        images, labels = data
        with tf.GradientTape() as tape:
            outputs = self(images, training=True) # forward pass 
            total_loss = custom_loss(labels, outputs)
        gradients = tape.gradient(total_loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        avg_loss.update_state(total_loss)
        return {"loss": avg_loss.result()}
model = custom_fit(inputs=inp, outputs=outputs)
model.compile(optimizer='adam')
history = model.fit(x, y, epochs=2_000)

PyTorch implementation:

In [ ]:
!wget -O SDT.py https://raw.githubusercontent.com/xuyxu/Soft-Decision-Tree/master/SDT.py
import SDT
import importlib
importlib.reload(SDT)

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split as tts

class dataset(Dataset):
 
    def __init__(self, data, seed = 42):
        X_train, X_valid, y_train, y_valid = tts(data.drop('Cover_Type', axis=1),
                                                 data['Cover_Type'],
                                                 random_state = seed)
        self.x_train=torch.tensor(X_train.values,
                                  dtype=torch.float32)
        self.y_train=torch.tensor(pd.get_dummies(y_train).values,
                                  dtype=torch.float32)
 
    def __len__(self):
        return len(self.y_train)
   
    def __getitem__(self,idx):
        return self.x_train[idx],self.y_train[idx]
    
import pandas as pd, numpy as np
df = pd.read_csv('../input/forest-cover-type-dataset/covtype.csv')
data = dataset(df.astype(np.float32))
dataloader = DataLoader(data, batch_size=64, shuffle=True)

from SDT import SDT
model = SDT(input_dim = len(X_train.columns),
            output_dim = len(np.unique(y_train)))

import torch.optim as optim
import torch.nn as nn
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
for epoch in range(10):

    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        inputs, labels = data
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        print(f'[Epoch: {epoch + 1}; Minibatch: {i + 1:5d}]. Loss: {loss.item():.3f}',
              end='\r')
    
    print('\n')

print('Finished Training')

### Deep Neural Network Initialization with Decision Trees (DJINN)

In [ ]:
# installation from github repo
!git clone https://github.com/LLNL/DJINN.git
!cd DJINN
!pip install -r requirements.txt
!pip install .

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from djinn import djinn

In [ ]:
# loading the breast cancer dataset
breast_cancer_data = load_breast_cancer()
X = breast_cancer_data.data
y = breast_cancer_data.target

X_train,X_test,y_train,y_test=train_test_split(X, y, test_size=0.25) 

In [ ]:
# dropout keep is the probability of keeping a neuron in dropout layers
djinn_model = djinn.DJINN_Classifier(n_trees=1, max_tree_depth=6, dropout_keep_prob=0.9)

# automatically search for optimal hyperparameters
optimal_params = djinn_model.get_hyperparameters(X_train, y_train)
batch_size = optimal_params['batch_size']
lr = optimal_params['learn_rate']
num_epochs = optimal_params['epochs']

# train
djinn_model.train(X_train, y_train,epochs=num_epochs,learn_rate=lr, batch_size=batch_size, 
              display_step=1,)

In [ ]:
# prediction and evaluation
from sklearn.metrics import roc_auc_score

preds = djinn_model.predict(X_test)
print(roc_auc_score(preds, y_test))

### Net-DNF

Custom model.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import backend as K
from keras import layers as L
import pandas as pd
from sklearn.model_selection import train_test_split as tts
data = pd.read_csv('../input/forest-cover-type-dataset/covtype.csv')
X_train, X_valid, y_train, y_valid = tts(data.drop('Cover_Type', axis=1), data['Cover_Type'])
NUM_LITERALS = 64
NUM_DISJ_ARGS = 32
AVG_NUM_CONJ_LITS = 16
NUM_DNNF_BLOCKS = 8

NUM_DISJ_ARGS_const = tf.constant(NUM_DISJ_ARGS, dtype=tf.float32)
def neural_or(x):
    out = K.tanh(K.sum(x, axis=1) + NUM_DISJ_ARGS_const - 1.5)
    return out
neural_or = L.Lambda(neural_or)

def neural_and(inputs):
    x, u = inputs
    u = tf.reshape(u, (NUM_LITERALS,1))
    return K.tanh(K.dot(x, u) - K.sum(u) + 1.5)
neural_and = L.Lambda(neural_and)

def DNNF(inp_layer):
    
    stimulus = tf.random.uniform((NUM_DISJ_ARGS, NUM_LITERALS))
    ratio = tf.constant(AVG_NUM_CONJ_LITS / NUM_LITERALS)
    masks = tf.cast(tf.math.less(stimulus, ratio), np.float32)

    literals = L.Dense(NUM_LITERALS, activation='tanh')(inp_layer)
    disj_args = []
    for i in range(NUM_DISJ_ARGS):
        disj_args.append(neural_and([literals, masks[i]]))
    disj_inp = L.Concatenate()(disj_args)
    disj = neural_or(disj_inp)
    return L.Reshape((1,))(disj)
    
def DNF_Net(input_dim, output_dim):
    
    inp = L.Input((input_dim,))
    dnnf_block_outs = []
    for i in range(NUM_DNNF_BLOCKS):
        dnnf_block_outs.append(DNNF(inp))
    concat = L.Concatenate()(dnnf_block_outs)
    out = L.Dense(output_dim, activation='softmax')(concat)
    
    return keras.models.Model(inputs=inp, outputs=out)
model = DNF_Net(len(X_train.columns), len(np.unique(y_train)))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
model.fit(X_train, y_train-1)

See https://github.com/amramabutbul/DisjunctiveNormalFormNet for official implementation.

---

## Boosting and Stacking Neural Networks

### GrowNet

We used a custom implementation from Yam Peleg by downloading the code from his Github Gist.

In [ ]:
# custom implementation for GrowNet
# code taken from https://gist.github.com/ypeleg/576c9c6470e7013ae4b4b7d16736947f

import tensorflow as tf
from copy import deepcopy
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, clone_model
from tensorflow.keras.layers import Input, Dropout, Dense, ReLU, BatchNormalization, Activation, Concatenate
import random

class DynamicNet(object):
    def __init__(self, c0 = None, lr = None, concat_input=False, additive_boosting=False, encoder_layers=None):
        self.models = []
        self.c0 = tf.Variable(np.float32(c0) if c0 is not None else random.uniform(0.0, 1.0))
        self.lr = lr
        self.boost_rate  = tf.Variable(lr if lr is not None else random.uniform(0.0, 1.0))
        self.concat_input = None
        self.additive_boosting = False
        self.encoder_layers = encoder_layers
    def freeze_all_networks(self):
        for model in self.models:
            for l in model.layers: l.trainable = False
    def unfreeze_all_networks(self):
        for model in self.models:
            for l in model.layers: l.trainable = True
    def add(self, model):
        last_activation = model.layers[-1].activation.__name__
        if last_activation in ['sigmoid', 'softmax']: model.layers[-1].activation = None
        if hasattr(model, 'optimizer') and model.optimizer is not None: self.loss = model.loss
        if hasattr(model, 'optimizer') and model.optimizer is not None: self.optimizer = model.optimizer
        if hasattr(model, 'optimizer') and model.optimizer is not None: self.lr = model.optimizer.lr if self.lr is None else self.lr
        if len(self.models) == 0:
            self.models = [model]
            self.full_model = self.models[-1]
            self.embed_full_model = self.models[-1]
        else: self.models.append(model)
        full_inp = Input(shape=self.models[0].input_shape[1:])
        out_orig = self.embed_full_model(full_inp)
        out = out_orig
        if self.concat_input: out = Concatenate()([out, full_inp])
        if len(self.models) > 1:
            if K.int_shape(out) != K.int_shape(self.models[-2].input): out = Dense(K.int_shape(self.models[-1].input)[-1])(out)
            new_out = self.models[-1](out)
        else: new_out = self.models[-1](full_inp)
        new_full_out = (self.c0) + (self.boost_rate * new_out)
        self.full_model = Model(full_inp, Activation(last_activation)(new_full_out))
        if self.encoder_layers is not None:
            if len(self.models) > 1: self.embed_full_model = Model(full_inp, Model(self.models[-1].input, self.models[-1].layers[self.encoder_layers].output)(out))
            else: self.embed_full_model = Model(full_inp, Model(self.models[-1].input, self.models[-1].layers[self.encoder_layers].output)(full_inp))
        else: self.embed_full_model = Model(full_inp, new_out)

    def fit(self, x_train, y_train, lr, w_decay=0.0, epochs=10, validation_data = None, **kwargs):
        if self.optimizer is None: optimizer = Adam(lr, decay=w_decay)
        else: optimizer = deepcopy(self.optimizer)
        self.full_model.compile(optimizer, self.loss)
        self.full_model.fit(x_train, y_train, epochs=epochs, validation_data = (x_train, y_train), **kwargs)
    def predict(self, x_train, **kwargs): return self.full_model.predict(x_train, **kwargs)


class GradientBoost(object):
    def __init__(self, base_model, lr = 1e-3, weight_decay = 1e-5, early_stopping_steps = 5, batch_size = 256, correct_epoch = 1, model_order = "second", n_boosting_rounds = 20 , boost_rate = 1.0, hidden_size=512, epochs_per_stage=1, encoder_layers=3):
        self.lr = lr
        self.base_model = base_model
        self.batch_size = batch_size
        self.boost_rate = boost_rate
        self.model_order = model_order
        self.hidden_size = hidden_size
        self.weight_decay = weight_decay
        self.num_nets = n_boosting_rounds
        self.encoder_layers = encoder_layers
        self.correct_epoch = correct_epoch
        self.epochs_per_stage = epochs_per_stage
        self.early_stopping_steps = early_stopping_steps

    def fit(self, x_train, y_train, validation_data = None, n_boosting_rounds=None, correct_epoch=None, epochs_per_stage=None, **kwargs):
        self.num_nets = n_boosting_rounds if n_boosting_rounds is not None else self.num_nets
        self.correct_epoch = correct_epoch if correct_epoch is not None else self.correct_epoch
        self.epochs_per_stage = epochs_per_stage if epochs_per_stage is not None else self.epochs_per_stage
        x_val , y_val = validation_data if validation_data is not None else (None, None)
        net_ensemble = DynamicNet(concat_input=True, encoder_layers=self.encoder_layers)
        lr = self.lr
        L2 = self.weight_decay
        for stage in range(self.num_nets):
            params = {}
            params["feat_d"] = x_train.shape[1]
            params["hidden_size"] = self.hidden_size
            new_model = clone_model(self.base_model)
            new_model.optimizer = deepcopy(self.base_model.optimizer)
            new_model.loss = self.base_model.loss
            net_ensemble.freeze_all_networks()
            net_ensemble.add(new_model)
            net_ensemble.fit(x_train, y_train, epochs=self.epochs_per_stage, lr=self.lr, validation_data = (x_train, y_train), **kwargs)
            lr_scaler = 2
            if stage != 0:
                if stage % 3 == 0: lr /= 2
                net_ensemble.unfreeze_all_networks()
                net_ensemble.fit(x_train, y_train, epochs=self.correct_epoch, lr=lr / lr_scaler, w_decay=L2, validation_data = (x_train, y_train))
        self.model = net_ensemble

    def predict(self, x_test, **kwargs): return self.model.predict(x_test, **kwargs)
    

In [ ]:
# we will use the california housing dataset for the example
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

import tensorflow as tf
import tensorflow.keras.callbacks as C
import tensorflow.keras.layers as L
import tensorflow.keras.models as M

data = fetch_california_housing()
X = data.data
y = data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# define the model architecture

inp = L.Input(X.shape[1])
x = L.BatchNormalization()(inp)
x = L.Dense(64, activation="swish")(x)
x = L.Dense(128, activation="swish")(x)
x = L.BatchNormalization()(x)
x = L.Dropout(0.25)(x)
x = L.Dense(32, activation="swish")(x)
x = L.BatchNormalization()(x)
out = L.Dense(1, activation='linear')(x)
model = M.Model(inp, out)
model.compile(tf.keras.optimizers.Adam(learning_rate=1e-3), 'mse')
model = GradientBoost(model, batch_size=4096, n_boosting_rounds=15, boost_rate = 1, epochs_per_stage=2)
model.fit(X_train, y_train,batch_size=4096)

In [ ]:
# prediction and evaluation
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, model.predict(X_test))

### XBNet

In [ ]:
# install from official Github repo
!pip install git+https://github.com/tusharsarkar3/XBNet.git

In [ ]:
from XBNet.training_utils import training,predict
from XBNet.models import XBNETClassifier
from XBNet.run import run_XBNET
import torch

# example dataset using sklearn's iris flower dataset
from sklearn.datasets import load_iris

raw_data = load_iris()
X, y = raw_data["data"], raw_data["target"]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
xbnet_model = XBNETClassifier(X_train,y_train, num_layers=3, num_layers_boosted=2,)

# define torch loss and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(xbnet_model.parameters(), lr=0.01)

xbnet_model, accuracy, loss, val_acc, val_loss = run_XBNET(X_train,X_test,y_train,y_test,
                                                           xbnet_model,criterion,optimizer,batch_size=16,
                                                           epochs=100, save=False)
# prediction
predict(xbnet_model, X_test)

---

## Distillation

DeepGBM does not have any easy to present implementation in a code notebook, refer to the repo for command line usages

### DeepGBM

See [this repository](https://github.com/motefly/DeepGBM).